In [2]:
from pathlib import Path
data = Path().parent.joinpath("../data").resolve()
fp = data.joinpath("titanic.csv").resolve()
fp

PosixPath('/home/a.fiorillo/coding/github.com/afiorillo/pandas-polars-comparison-presentation/data/titanic.csv')

# Loading a File

With each library, let's show what loading a file looks like.

In [3]:
import pandas as pd
df_pd = pd.read_csv(fp)
df_pd.head()

,Passengerid,Age,Fare,Sex,sibsp,zero,zero.1,zero.2,zero.3,zero.4,...,zero.12,zero.13,zero.14,Pclass,zero.15,zero.16,Embarked,zero.17,zero.18,2urvived
0,1,22.0,7.2500,0,1,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
1,2,38.0,71.2833,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0.0,0,0,1
2,3,26.0,7.9250,1,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,1
3,4,35.0,53.1000,1,1,0,0,0,0,0,...,0,0,0,1,0,0,2.0,0,0,1
4,5,35.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0


In [4]:
import polars as pl
df_pl = pl.read_csv(fp)
df_pl.head()

Passengerid,Age,Fare,Sex,sibsp,zero,zero_duplicated_0,zero_duplicated_1,zero_duplicated_2,zero_duplicated_3,zero_duplicated_4,zero_duplicated_5,Parch,zero_duplicated_6,zero_duplicated_7,zero_duplicated_8,zero_duplicated_9,zero_duplicated_10,zero_duplicated_11,zero_duplicated_12,zero_duplicated_13,Pclass,zero_duplicated_14,zero_duplicated_15,Embarked,zero_duplicated_16,zero_duplicated_17,2urvived
i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,22.0,7.25,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,2,0,0,0
2,38.0,71.2833,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,26.0,7.925,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,2,0,0,1
4,35.0,53.1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,1
5,35.0,8.05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,2,0,0,0


# Simple Analysis

With each library, let's answer the question "what is the distribution of ages of the female passengers?" Min, median, max, etc.

In [5]:
# Make a mask, apply it, select the column, and use `.describe()`
mask = df_pd['Sex'] == 1
masked_ages = df_pd[mask]['Age']
masked_ages.describe()

count    466.000000
mean      28.572082
std       13.300776
min        0.170000
25%       21.000000
50%       28.000000
75%       35.000000
max       76.000000
Name: Age, dtype: float64

In [6]:
# Make a filter, select the column, and use `.describe()`
mask = df_pl.filter(pl.col("Sex") == 1)
masked_ages = mask.select(pl.col("Age"))
masked_ages.describe()

statistic,Age
str,f64
"""count""",466.0
"""null_count""",0.0
"""mean""",28.572082
"""std""",13.300776
"""min""",0.17
"""25%""",21.0
"""50%""",28.0
"""75%""",35.0
"""max""",76.0


# Data Change

To do some more interesting computations, we will use a larger dataset with GIS. [The Nepalese Climate](https://www.kaggle.com/datasets/saimondahal/nepal-daily-climate-1981-2019) data will work.

We can take the opportunity to benchmark IO.

In [7]:
from pathlib import Path
data = Path().parent.joinpath("../data").resolve()
fp = data.joinpath("nepal_climate.csv").resolve()
fp

PosixPath('/home/a.fiorillo/coding/github.com/afiorillo/pandas-polars-comparison-presentation/data/nepal_climate.csv')

In [8]:
df_pd = pd.read_csv(fp)
%timeit df_pd = pd.read_csv(fp)

df_pd.head()

793 ms ± 29.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


,Unnamed: 0,Date,District,Latitude,Longitude,Precip,Pressure,Humidity_2m,RH_2m,Temp_2m,...,TempRange_2m,EarthSkinTemp,WindSpeed_10m,MaxWindSpeed_10m,MinWindSpeed_10m,WindSpeedRange_10m,WindSpeed_50m,MaxWindSpeed_50m,MinWindSpeed_50m,WindSpeedRange_50m
0,0,1981-01-01,Arghakhanchi,27.9,83.2,0.00,93.51,4.81,45.41,13.89,...,10.89,11.32,1.89,3.83,0.69,3.14,2.41,4.12,0.73,3.39
1,1,1981-01-02,Arghakhanchi,27.9,83.2,0.00,93.59,4.94,46.78,13.84,...,11.17,11.44,1.72,2.60,1.09,1.50,2.25,3.30,0.96,2.34
2,2,1981-01-03,Arghakhanchi,27.9,83.2,0.03,93.55,5.22,47.91,14.33,...,9.93,12.24,1.80,2.80,0.48,2.32,2.32,3.54,0.39,3.15
3,3,1981-01-04,Arghakhanchi,27.9,83.2,0.02,93.49,5.36,50.83,13.82,...,10.41,12.17,2.18,3.54,1.06,2.49,2.90,4.05,0.93,3.12
4,4,1981-01-05,Arghakhanchi,27.9,83.2,1.84,93.49,5.84,55.55,13.76,...,10.53,12.32,1.96,2.70,0.69,2.02,2.74,4.64,0.96,3.68


In [9]:
df_pl = pl.read_csv(fp)
%timeit df_pl = pl.read_csv(fp)

df_pl.head()

102 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


,Date,District,Latitude,Longitude,Precip,Pressure,Humidity_2m,RH_2m,Temp_2m,WetBulbTemp_2m,MaxTemp_2m,MinTemp_2m,TempRange_2m,EarthSkinTemp,WindSpeed_10m,MaxWindSpeed_10m,MinWindSpeed_10m,WindSpeedRange_10m,WindSpeed_50m,MaxWindSpeed_50m,MinWindSpeed_50m,WindSpeedRange_50m
i64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,"""1981-01-01""","""Arghakhanchi""",27.9,83.2,0.0,93.51,4.81,45.41,13.89,2.15,20.82,9.94,10.89,11.32,1.89,3.83,0.69,3.14,2.41,4.12,0.73,3.39
1,"""1981-01-02""","""Arghakhanchi""",27.9,83.2,0.0,93.59,4.94,46.78,13.84,2.54,20.7,9.54,11.17,11.44,1.72,2.6,1.09,1.5,2.25,3.3,0.96,2.34
2,"""1981-01-03""","""Arghakhanchi""",27.9,83.2,0.03,93.55,5.22,47.91,14.33,3.32,20.71,10.78,9.93,12.24,1.8,2.8,0.48,2.32,2.32,3.54,0.39,3.15
3,"""1981-01-04""","""Arghakhanchi""",27.9,83.2,0.02,93.49,5.36,50.83,13.82,3.73,20.43,10.02,10.41,12.17,2.18,3.54,1.06,2.49,2.9,4.05,0.93,3.12
4,"""1981-01-05""","""Arghakhanchi""",27.9,83.2,1.84,93.49,5.84,55.55,13.76,4.93,19.62,9.08,10.53,12.32,1.96,2.7,0.69,2.02,2.74,4.64,0.96,3.68


In [10]:
df_pd.shape

(883128, 23)

# An Aggregation

Let's make the index of the DataFrames a tuple of (lat, lng) and find the date of the record maximum wind speed for each location.

In [12]:
def date_of_max_wind_speed_pd(df):
    return ((df
        # sort the rows from highest to lowest wind speed
        .sort_values('MaxWindSpeed_50m', ascending=False)
        # group by the lat/lng pair
        .groupby(['Latitude', 'Longitude'])
        # taking the first for each pair means the max, because we sorted
        .first()
    )
     # we have to do another sort or else the lat/lng pairs will act as index
     .sort_values('MaxWindSpeed_50m', ascending=False)
     # and finally we want only a few columns to display
     .filter(['Date', 'Latitude', 'Longitude', 'District', 'MaxWindSpeed_50m'])
    )

%timeit date_of_max_wind_speed_pd(df_pd)

date_of_max_wind_speed_pd(df_pd)

365 ms ± 25.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


,,Date,District,MaxWindSpeed_50m
Latitude,Longitude,,,
30.0,81.8,1994-02-21,Humla,22.85
29.5,82.1,1994-02-21,Mugu,20.18
28.0,86.8,2012-02-08,Solukhumbu,19.70
28.6,84.2,2012-02-08,Manang,19.47
26.8,85.3,1994-06-29,Routahat,17.76
...,...,...,...,...
28.3,83.6,1992-03-24,Baglung,9.84
28.6,82.4,1983-03-20,Rukum,9.82
28.2,84.0,1983-03-12,Kaski,8.94


In [22]:
def date_of_max_wind_speed_pl(df):
    return (df
        .sort("MaxWindSpeed_50m", descending=True)
        .group_by(["Latitude", "Longitude"])
        .head(1)
    ).sort("MaxWindSpeed_50m", descending=True).select(["Date", 'Latitude', 'Longitude', 'District', 'MaxWindSpeed_50m'])

%timeit date_of_max_wind_speed_pl(df_pl)

date_of_max_wind_speed_pl(df_pl)

117 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Date,Latitude,Longitude,District,MaxWindSpeed_50m
str,f64,f64,str,f64
"""1994-02-21""",30.0,81.8,"""Humla""",22.85
"""1994-02-21""",29.5,82.1,"""Mugu""",20.18
"""2012-02-08""",28.0,86.8,"""Solukhumbu""",19.7
"""2012-02-08""",28.6,84.2,"""Manang""",19.47
"""1994-06-29""",26.8,85.3,"""Routahat""",17.76
"""1992-03-24""",26.6,86.8,"""Saptari""",17.74
"""1992-03-24""",26.9,86.5,"""Udayapur""",17.74
"""1994-02-21""",29.0,82.9,"""Dolpa""",17.57
"""2008-09-19""",28.3,81.4,"""Bardiya""",17.1


# A User Defined Function

It's not uncommon when an aggregation requires some kind of custom function. Here we will define a `point_in_polygon(lat, lng, polygon)` function which we want to apply for each row of the Dataframe. With it, we can find the record high precipitation for all points only within a given polygon.

In [11]:
nepalese_border_s = """{"type": "Feature", "properties": {}, "geometry": {"coordinates": [[[88.15528458405566, 27.518715994978322], [88.40167327056588, 28.133297098638067], [85.86197450193436, 28.478156145828834], [83.23382851250426, 29.884633978458467], [81.49015473105669, 30.632227143031656], [81.15531882374927, 30.05976298407485], [82.0145203972171, 29.43446195878984], [83.54339378529846, 28.57806668825164], [85.31233820126067, 27.915798584320527], [86.68958778225954, 27.535523345602627], [88.15528458405566, 27.518715994978322]]], "type": "Polygon"}}"""

from shapely import from_geojson
import folium

nepalese_border = from_geojson(nepalese_border_s)
map = folium.Map(location=[28.453, 84.199], zoom_start=7, tiles="CartoDB Positron")
folium.GeoJson(nepalese_border).add_to(map)
map

In [29]:
border_point = from_geojson("""{"type": "Feature", "properties": {}, "geometry": {"coordinates": [83.95730917416023, 29.1840613886463], "type": "Point"}}""")
kathmandu = from_geojson("""{"type": "Feature", "properties": {}, "geometry": {"coordinates": [85.31993342931895, 27.70594391330421], "type": "Point"}}""")

folium.GeoJson(
    border_point,
    marker=folium.Circle(radius=10000, fill_color="green", fill_opacity=0.8, color="black", weight=1)
).add_to(map)
folium.GeoJson(
    kathmandu,
    marker=folium.Circle(radius=10000, fill_color="orange", fill_opacity=0.8, color="black", weight=1)
).add_to(map)
map

In [89]:
from shapely import Point, Polygon

def point_in_polygon(point: Point, polygon: Polygon) -> bool:
    return polygon.contains(point)

print("Expect True:", point_in_polygon(border_point, nepalese_border))
print("Expect False:", point_in_polygon(kathmandu, nepalese_border))

Expect True: True
Expect False: False


In [90]:
def row_within_border(row):
    point = Point([row['Longitude'], row['Latitude']])
    return nepalese_border.contains(point)

def naive_max_precip_within_border_region_pd(df):
    mask = df.apply(row_within_border, axis=1)
    rows_within_border = df[mask]
    return ((rows_within_border
        # sort the rows from highest to lowest wind speed
        .sort_values('Precip', ascending=False)
        # group by the lat/lng pair
        .groupby(['Latitude', 'Longitude'])
        # taking the first for each pair means the max, because we sorted
        .first()
    )
     # we have to do another sort or else the lat/lng pairs will act as index
     .sort_values('Precip', ascending=False)
     # and finally we want only a few columns to display
     .filter(['Date', 'Latitude', 'Longitude', 'District', 'Precip'])
    )

%timeit naive_max_precip_within_border_region_pd(df_pd)

naive_max_precip_within_border_region_pd(df_pd)

29.5 s ± 844 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


,,Date,District,Precip
Latitude,Longitude,,,
28.3,84.4,2016-07-26,Lamjung,93.64
29.3,82.3,1983-09-11,Jumla,93.40
29.0,82.9,1983-09-11,Dolpa,90.35
28.0,86.8,1981-06-29,Solukhumbu,84.98
28.6,84.2,2016-07-26,Manang,82.60
28.1,85.3,1981-06-29,Rasuwa,78.82
28.9,83.8,2015-07-16,Mustang,77.99
29.5,82.1,1983-09-11,Mugu,75.81
30.0,81.8,1983-09-11,Humla,58.53


In [125]:
def row_within_border(row):
    point = Point([row[4], row[3]])
    return nepalese_border.contains(point)

def naive_max_precip_within_border_region_pl(df):    
    mask = df.map_rows(row_within_border)
    rows_within_border = df.filter(mask.get_column("map"))
    return (rows_within_border
        .sort("Precip", descending=True)
        .group_by(["Latitude", "Longitude"])
        .head(1)
    ).sort("Precip", descending=True).select(["Date", 'Latitude', 'Longitude', 'District', 'Precip'])
%timeit naive_max_precip_within_border_region_pl(df_pl)

naive_max_precip_within_border_region_pl(df_pl)

28.4 s ± 1.53 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Date,Latitude,Longitude,District,Precip
str,f64,f64,str,f64
"""2016-07-26""",28.3,84.4,"""Lamjung""",93.64
"""1983-09-11""",29.3,82.3,"""Jumla""",93.4
"""1983-09-11""",29.0,82.9,"""Dolpa""",90.35
"""1981-06-29""",28.0,86.8,"""Solukhumbu""",84.98
"""2016-07-26""",28.6,84.2,"""Manang""",82.6
"""1981-06-29""",28.1,85.3,"""Rasuwa""",78.82
"""2015-07-16""",28.9,83.8,"""Mustang""",77.99
"""1983-09-11""",29.5,82.1,"""Mugu""",75.81
"""1983-09-11""",30.0,81.8,"""Humla""",58.53


## UDFs are Bad!

The previous UDF is very slow, and obviously so. Instead, it is almost always faster to find a vectorized implementation -- something which moves all the data to be checked as a single array and reduces the computation into linear algebra.

In [51]:
import json
import numpy as np
from matplotlib.path import Path

border_arr = np.array(json.loads(nepalese_border_s)['geometry']['coordinates'][0])
p = Path(border_arr)

print("Expect True:", p.contains_point(list(border_point.coords)[0]))
print("Expect False:", p.contains_point(list(kathmandu.coords)[0]))

def points_within_border(lng_lat_arr):
    return p.contains_points(lng_lat_arr)

test_points = np.array([
    list(border_point.coords)[0],
    list(kathmandu.coords)[0],
])
print('Expect [True, False]:', points_within_border(test_points))

Expect True: True
Expect False: False
Expect [True, False]: [ True False]


In [47]:
def vector_max_precip_within_border_region_pd(df):
    test_points = df.filter(['Longitude', 'Latitude']).to_numpy()
    mask = points_within_border(test_points)
    # and the rest is the same as before
    rows_within_border = df[mask]
    return ((rows_within_border
        # sort the rows from highest to lowest wind speed
        .sort_values('Precip', ascending=False)
        # group by the lat/lng pair
        .groupby(['Latitude', 'Longitude'])
        # taking the first for each pair means the max, because we sorted
        .first()
    )
     # we have to do another sort or else the lat/lng pairs will act as index
     .sort_values('Precip', ascending=False)
     # and finally we want only a few columns to display
     .filter(['Date', 'Latitude', 'Longitude', 'District', 'Precip'])
    )

%timeit vector_max_precip_within_border_region_pd(df_pd)

vector_max_precip_within_border_region_pd(df_pd)

66.4 ms ± 243 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


,,Date,District,Precip
Latitude,Longitude,,,
28.3,84.4,2016-07-26,Lamjung,93.64
29.3,82.3,1983-09-11,Jumla,93.40
29.0,82.9,1983-09-11,Dolpa,90.35
28.0,86.8,1981-06-29,Solukhumbu,84.98
28.6,84.2,2016-07-26,Manang,82.60
28.1,85.3,1981-06-29,Rasuwa,78.82
28.9,83.8,2015-07-16,Mustang,77.99
29.5,82.1,1983-09-11,Mugu,75.81
30.0,81.8,1983-09-11,Humla,58.53


In [50]:
def vector_max_precip_within_border_region_pl(df):   
    test_points = df.select(['Longitude', 'Latitude']).to_numpy()
    mask = points_within_border(test_points)
    # and the rest is as before
    rows_within_border = df.filter(mask)
    return (rows_within_border
        .sort("Precip", descending=True)
        .group_by(["Latitude", "Longitude"])
        .head(1)
    ).sort("Precip", descending=True).select(["Date", 'Latitude', 'Longitude', 'District', 'Precip'])
    
%timeit vector_max_precip_within_border_region_pl(df_pl)

vector_max_precip_within_border_region_pl(df_pl)

40 ms ± 432 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Date,Latitude,Longitude,District,Precip
str,f64,f64,str,f64
"""2016-07-26""",28.3,84.4,"""Lamjung""",93.64
"""1983-09-11""",29.3,82.3,"""Jumla""",93.4
"""1983-09-11""",29.0,82.9,"""Dolpa""",90.35
"""1981-06-29""",28.0,86.8,"""Solukhumbu""",84.98
"""2016-07-26""",28.6,84.2,"""Manang""",82.6
"""1981-06-29""",28.1,85.3,"""Rasuwa""",78.82
"""2015-07-16""",28.9,83.8,"""Mustang""",77.99
"""1983-09-11""",29.5,82.1,"""Mugu""",75.81
"""1983-09-11""",30.0,81.8,"""Humla""",58.53
